In [1]:
#assumptions that will break if we use MCTS for play traces in this code
#1. Turn wont be either 0 or 1
#2. Cant just copy the final two rows to extend round number to maxNoOfRounds

#comments: So play traces for a given strategy will change dependent on the opponent's strategy, e.g. evolution of 
#bigmoney deck is different if it is played vs another bigmoney startegy as opposed to say a witch strategy which will 
#introduce curse cards into the bigmoney trace

In [28]:
import pandas as pd
import pdb
import math
import itertools
pd.set_option('display.max_rows', 100)
noPlayers = 2

In [3]:
data  = pd.read_csv("data/featureslogfile.txt", sep = '\t')

In [4]:
print(data.columns)

Index(['GameID', 'Player', 'Round', 'Turn', 'CurrentScore', 'SCORE',
       'SCORE_ADV', 'ORDINAL', 'OUR_TURN', 'HAS_WON', 'FINAL_ORD', 'ROUND',
       'CURSE', 'ESTATE', 'DUCHY', 'PROVINCE', 'COPPER', 'SILVER', 'GOLD',
       'CELLAR', 'CHAPEL', 'MOAT', 'HARBINGER', 'MERCHANT', 'VASSAL',
       'VILLAGE', 'WORKSHOP', 'BUREAUCRAT', 'GARDENS', 'MILITIA',
       'MONEYLENDER', 'POACHER', 'REMODEL', 'SMITHY', 'THRONE_ROOM', 'BANDIT',
       'COUNCIL_ROOM', 'FESTIVAL', 'LABORATORY', 'LIBRARY', 'MARKET', 'MINE',
       'SENTRY', 'WITCH', 'ARTISAN', 'PlayerCount', 'TotalRounds',
       'ActionScore', 'Win', 'Ordinal', 'FinalScore'],
      dtype='object')


In [5]:
#check number of games
print(data['GameID'].unique())

[ 2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18. 19.
 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36. 37.
 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51.]


In [6]:
#note that Player field swaps round half way through the tournament when there are two players. Also the GameID starts
#at 2 for some reason
noGamesPerMatchUp = 25
def assignName(gameID, player, noGamesPerMatchUp):
    name = ""
    if gameID < (noGamesPerMatchUp + 2):
        if player == 0:
            name = 'BigMoneyWithGardens'
        else:
            name = 'DoubleWitch'
    else:
        if player == 1:
            name = 'BigMoneyWithGardens'
        else:
            name = 'DoubleWitch'
    return name

In [7]:
#add label for player types
data['PlayerName'] = data.apply(lambda df: assignName(df['GameID'], df['Player'], noGamesPerMatchUp), axis = 1)

In [8]:
print(data['GameID'].unique())

[ 2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18. 19.
 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36. 37.
 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51.]


In [9]:
#we can only record events each turn so we need to use the final turn in each round for each player to determine their
#deck at the end of the round
print(data['Turn'].unique())
data = data[data['Turn'] == 1]
print(data['Turn'].unique())

[0. 1.]
[1.]


In [23]:
#extract columns of interest 
card_types_in_supply = ['ARTISAN', 'BANDIT', 'BUREAUCRAT', 'CHAPEL', 'FESTIVAL', 'GARDENS', 'SENTRY',
                        'THRONE ROOM', 'WITCH', 'CURSE','WORKSHOP','PROVINCE', 'DUCHY', 'ESTATE', 'GOLD', 'SILVER', 'COPPER']
card_types_BigMoneyWithGardens_vs_DoubleWitch = ['GARDENS', 'WITCH', 'CURSE', 'PROVINCE', 'DUCHY', 'ESTATE', 'GOLD', 'SILVER', 'COPPER']

In [11]:
#define columns of interest
cols = ['PlayerName', 'GameID', 'Round', 'CurrentScore']
cols = cols + card_types_BigMoneyWithGardens_vs_DoubleWitch

In [12]:
#extract all BigMoneyWithGardens play-traces
Traces_BigMoneyWithGardens = data[data['PlayerName'] == 'BigMoneyWithGardens'].loc[:, cols]
print(Traces_BigMoneyWithGardens)

               PlayerName  GameID  Round  CurrentScore  GARDENS  WITCH  CURSE  \
2     BigMoneyWithGardens     2.0    0.0           3.0      0.0    0.0    0.0   
6     BigMoneyWithGardens     2.0    1.0           3.0      0.0    0.0    0.0   
10    BigMoneyWithGardens     2.0    2.0           3.0      0.0    0.0    0.0   
14    BigMoneyWithGardens     2.0    3.0           3.0      0.0    0.0    0.0   
18    BigMoneyWithGardens     2.0    4.0           3.0      0.0    0.0    0.0   
...                   ...     ...    ...           ...      ...    ...    ...   
4337  BigMoneyWithGardens    51.0   14.0          -1.0      0.0    0.0    4.0   
4341  BigMoneyWithGardens    51.0   15.0          -1.0      0.0    0.0    4.0   
4345  BigMoneyWithGardens    51.0   16.0           4.0      0.0    0.0    5.0   
4349  BigMoneyWithGardens    51.0   17.0          10.0      0.0    0.0    5.0   
4353  BigMoneyWithGardens    51.0   18.0          15.0      0.0    0.0    6.0   

      PROVINCE  DUCHY  ESTA

In [13]:
#some sense checks
print(Traces_BigMoneyWithGardens['WITCH'].unique())
print(Traces_BigMoneyWithGardens['GameID'].unique())

[0.]
[ 2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18. 19.
 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36. 37.
 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51.]


In [14]:
#extract all DoubleWitch play-traces
Traces_DoubleWitch = data[data['PlayerName'] == 'DoubleWitch'].loc[:, cols]
print(Traces_DoubleWitch)

       PlayerName  GameID  Round  CurrentScore  GARDENS  WITCH  CURSE  \
3     DoubleWitch     2.0    0.0           3.0      0.0    0.0    0.0   
7     DoubleWitch     2.0    1.0           3.0      0.0    0.0    0.0   
11    DoubleWitch     2.0    2.0           3.0      0.0    1.0    0.0   
15    DoubleWitch     2.0    3.0           3.0      0.0    2.0    0.0   
19    DoubleWitch     2.0    4.0           3.0      0.0    2.0    0.0   
...           ...     ...    ...           ...      ...    ...    ...   
4336  DoubleWitch    51.0   14.0          15.0      0.0    2.0    0.0   
4340  DoubleWitch    51.0   15.0          15.0      0.0    2.0    0.0   
4344  DoubleWitch    51.0   16.0          21.0      0.0    2.0    0.0   
4348  DoubleWitch    51.0   17.0          21.0      0.0    2.0    0.0   
4352  DoubleWitch    51.0   18.0          27.0      0.0    2.0    0.0   

      PROVINCE  DUCHY  ESTATE  GOLD  SILVER  COPPER  
3          0.0    0.0     3.0   0.0     1.0     7.0  
7          0.0 

In [15]:
#all play taces in a single data frame
traces = data.loc[:, cols]
print(traces)

               PlayerName  GameID  Round  CurrentScore  GARDENS  WITCH  CURSE  \
2     BigMoneyWithGardens     2.0    0.0           3.0      0.0    0.0    0.0   
3             DoubleWitch     2.0    0.0           3.0      0.0    0.0    0.0   
6     BigMoneyWithGardens     2.0    1.0           3.0      0.0    0.0    0.0   
7             DoubleWitch     2.0    1.0           3.0      0.0    0.0    0.0   
10    BigMoneyWithGardens     2.0    2.0           3.0      0.0    0.0    0.0   
...                   ...     ...    ...           ...      ...    ...    ...   
4345  BigMoneyWithGardens    51.0   16.0           4.0      0.0    0.0    5.0   
4348          DoubleWitch    51.0   17.0          21.0      0.0    2.0    0.0   
4349  BigMoneyWithGardens    51.0   17.0          10.0      0.0    0.0    5.0   
4352          DoubleWitch    51.0   18.0          27.0      0.0    2.0    0.0   
4353  BigMoneyWithGardens    51.0   18.0          15.0      0.0    0.0    6.0   

      PROVINCE  DUCHY  ESTA

In [16]:
#we need to make sure all games have the same number of rounds in them
gameLengths = traces.groupby(['GameID'])['Round'].max()
maxNoOfRounds = int(gameLengths.max()) + 1 #round counter starts at zero
print(maxNoOfRounds)

25


In [17]:
def copy_final_deck_at_game_end(group, roundMax, noPlayers):
    #This function repeatedly copies the final decks of two players at the game end, so that the game is extended to 
    #have roundMax rounds
    final_round = int(group['Round'].max())
    if (roundMax-1) == final_round:
        #in this case we dont need to extend the play trace
        return group
    else:
        final_row_copy = pd.concat([group.iloc[-noPlayers:]] * ((roundMax-1) - final_round), ignore_index=True)
        #we need to update the Round counter so that every other row it increments by one
        final_row_copy['Round'] = [final_round + 1 + i // 2 for i in range(((roundMax-1) - final_round)*2)]
        return pd.concat([group, final_row_copy], ignore_index=True)

In [18]:
extended_traces = traces.groupby('GameID').apply(copy_final_deck_at_game_end, maxNoOfRounds, noPlayers = 2).reset_index(drop = True)
print(extended_traces)

               PlayerName  GameID  Round  CurrentScore  GARDENS  WITCH  CURSE  \
0     BigMoneyWithGardens     2.0    0.0           3.0      0.0    0.0    0.0   
1             DoubleWitch     2.0    0.0           3.0      0.0    0.0    0.0   
2     BigMoneyWithGardens     2.0    1.0           3.0      0.0    0.0    0.0   
3             DoubleWitch     2.0    1.0           3.0      0.0    0.0    0.0   
4     BigMoneyWithGardens     2.0    2.0           3.0      0.0    0.0    0.0   
...                   ...     ...    ...           ...      ...    ...    ...   
2495  BigMoneyWithGardens    51.0   22.0          15.0      0.0    0.0    6.0   
2496          DoubleWitch    51.0   23.0          27.0      0.0    2.0    0.0   
2497  BigMoneyWithGardens    51.0   23.0          15.0      0.0    0.0    6.0   
2498          DoubleWitch    51.0   24.0          27.0      0.0    2.0    0.0   
2499  BigMoneyWithGardens    51.0   24.0          15.0      0.0    0.0    6.0   

      PROVINCE  DUCHY  ESTA

In [19]:
noOfGames = extended_traces['GameID'].unique().size
noOfSymmetricGameCombos = int(noOfGames *(noOfGames -1)/2.0) #dont include diagonal elements as we include comparing play traces
#within a given game via multiplying by six below
#For each game combo, you can compare play traces between the games (gives 4 combos) plus within the two games (gives another 2 traces)
noOfPairWiseDistances = noOfSymmetricGameCombos * 6
print(noOfSymmetricGameCombos)
print(noOfPairWiseDistances)

1225
7350


In [20]:
#next we need to collect together all pairwise combinations of traces so that we can compute a distance measure
# Step 1: Create all symmetric combinations of GameIDs 
unique_values = extended_traces['GameID'].unique()
symmetric_combinations = list(itertools.combinations(unique_values, 2))

# Step 2: Create a list of DataFrames for each symmetric combination
list_dfs = []
for combination in symmetric_combinations:
    #extract traces for each game ID and strategy combination
    GameID_1, GameID_2 = combination
    keys = []
    count = 0
    for player in ['BigMoneyWithGardens', 'DoubleWitch']:
        for game in [GameID_1, GameID_2]:
            keys.append((game,player))

    #then for each pair of keys in our dictionary we need to merge the dataframes
    keys_combos= list(itertools.combinations(keys, 2))
    for key_pair in keys_combos:
        key1, key2 = key_pair
        game1, player1 = key1
        game2, player2 = key2
        df_A = extended_traces[(extended_traces['GameID'] == game1) & (extended_traces['PlayerName'] == player1)]
        df_B = extended_traces[(extended_traces['GameID'] == game2) & (extended_traces['PlayerName'] == player2)]
        df_1 = df_A.rename(columns={'PlayerName' : 'PlayerName_1', 'GameID': 'GameID_1', 
                                                     'CurrentScore': 'CurrentScore_1', 'GARDENS': 'GARDENS_1',
                                                     'WITCH': 'WITCH_1', 'CURSE': 'CURSE_1','PROVINCE': 'PROVINCE_1',
                                                     'DUCHY':'DUCHY_1', 'ESTATE': 'ESTATE_1', 'GOLD': 'GOLD_1', 
                                                     'SILVER':'SILVER_1', 'COPPER':'COPPER_1'})
        df_2 = df_B.rename(columns={'PlayerName' : 'PlayerName_2', 'GameID': 'GameID_2', 
                                                     'CurrentScore': 'CurrentScore_2', 'GARDENS': 'GARDENS_2',
                                                     'WITCH': 'WITCH_2', 'CURSE': 'CURSE_2','PROVINCE': 'PROVINCE_2',
                                                     'DUCHY':'DUCHY_2', 'ESTATE': 'ESTATE_2', 'GOLD': 'GOLD_2', 
                                                     'SILVER':'SILVER_2', 'COPPER':'COPPER_2'})
        #next align play traces by linking the common round column
        df_3 = pd.merge(df_1, df_2, on = 'Round')
        list_dfs.append(df_3)

#finally we need to append togther all the datafarmes in our list
sym_trace_combos = pd.concat(list_dfs, ignore_index=True)

In [21]:
#the no of rows in our symmetric trace combos should be given by noOfPairwiseDistances * maxNoRounds
print(noOfPairWiseDistances * maxNoOfRounds)
sym_trace_combos.shape

183750


(183750, 25)

In [26]:
#next step is to calculate the distance between the decks at each round for each given pair of play traces
def deck_dist_measure(playTracePairForSingleRound, cardTypes):
    dist = 0
    for card in cardTypes:
        col1 = card + "_1"
        col2 = card + "_2"
        dist += (playTracePairForSingleRound[col1]-playTracePairForSingleRound[col2])**2
    dist = math.sqrt(dist)/math.sqrt(len(cardTypes))
    return dist

In [32]:
sym_trace_combos['dist'] = sym_trace_combos.apply(lambda row: deck_dist_measure(row, card_types_BigMoneyWithGardens_vs_DoubleWitch), axis = 1)

In [35]:
#do some sense checks on distance calculation
#firtsly look at averages of distaace measure between similalr startegies vs between different strategies